# `*expression`/`**expression`和函数参数


## `*expression`和`**expression`

`*expression`和`**expression`能够在函数调用、列表推导或生成器表达式中提供unpacking操作，其中`*`为迭代器对象的unpacking操作符，`**`为`dict`类型对象的unpacking操作符。

`*expression`应用场景：

1. 函数定义时表示可变positional形参列表
2. 作为赋值语句的左值，对右值进行unpacking操作
3. 函数调用时对`list`、`tuple`等可迭代对象进行unpacking操作，提供positional实参
4. 在`list`或`tuple`中对可迭代对象元素进行unpacking操作

其中1与2，3与4分别有类似之处。

`**expression`应用场景：

1. 函数定义时表示可变keyword形参列表
2. 函数调用时对`dict`类型进行unpacking操作，提供keyword实参
3. 在`dict`对象中，对嵌套的`dict`对象进行unpacking操作

其中2与3有类似之处。

### `list`、`tuple`、`set`以及`dict`中的可迭代对象的unpacking操作

`*expression`可用于对可迭代对象进行unpacking操作，包括`tuple`、`list`、`string`、`file`、迭代器与生成器，和`**expression`可用于对`dict`进行unpacking操作。

这类unpacking操作只能出现在`tuple`、`list`、`set`和`dict`对象中。*据此推测，在`tuple`、`list`、`set`和`dict`对象中，对可迭代对象进行unpacking操作后，实际上是将其转换为postional参数或keyword参数，并传递给了所在容器的构造器。*

In [3]:
from collections import Iterable
isinstance({'a': 1, 'b': 2},Iterable)

True

In [ ]:
# 在`tuple`中
*range(4), 4

(0, 1, 2, 3, 4)

In [2]:
a = [1, 2, 3]
b = a
a.append(4)
b == a
b

[1, 2, 3, 4]

In [14]:
list((1,2,3))

[1, 2, 3]

In [3]:
[(1, 3, 4)]

[(1, 3, 4)]

In [2]:
# 在list中
[*range(4), 4]

[0, 1, 2, 3, 4]

In [3]:
# 在`set`中
{*range(4), 4}

{0, 1, 2, 3, 4}

In [4]:
# 在dict中
{'x': 1, **{'y': 2, 'z': 3}}

{'x': 1, 'y': 2, 'z': 3}

`*expression`与`**expression`不能对一个单独的可迭代对象进行unpacking操作。

In [4]:
# wrong
# *range(4)

同一个容器中支持多个`*expression`和`**expression`。

In [15]:
[1, 2, *[3, 4], *[5], *(6, 7)]

[1, 2, 3, 4, 5, 6, 7]

In [16]:
(1, *[2, 3], *{'a': 1})

(1, 2, 3, 'a')

In [17]:
1, *[2, 3], *{'a': 1}

(1, 2, 3, 'a')

In [18]:
{'a': 1, **{'a': 2, 'c': 3}, **{'c': 'new 3', 'd': 4}}

{'a': 2, 'c': 'new 3', 'd': 4}

### 赋值语句中左值变量的unpacking操作

对于左值表达式包含多个变量（即左值为包含多个变量的一个`tuple`或`list`）的赋值语句，左值中包含`*expression`符号的变量为`*expression`变量，而其余的变量为常规变量，在保证常规变量都被赋值的情况下，右值剩余的变量将以一个`list`的形式赋值给对应`*expression`变量。

由于当`*expression`超过一个时，unpacking操作将出现歧义，因此赋值语句的左值至多包含一个`*expression`。

`*expression`只能在赋值语句中作为左值变量（或在函数定义与调用中）使用，在任何其他地方使用均为错误。

`**expression`在这种场景下并不适用，因此无法对右值的dict对象进行unpacking操作。

In [10]:
# 这里a, *b, c与(a, *b, c)或[a, *b, c]是等价的
a, *b, c = range(5)
print(a)
print(b)
print(c)

0
[1, 2, 3]
4


赋值语句中，`*expression`不能作为唯一一个左值变量。

In [11]:
# 错误
# *a = range(5)

# 正确
*a, = range(5)
a

[0, 1, 2, 3, 4]

In [13]:
def fixed_keyword_starred_unpacking_func(*, a, b, c):
    print(a, b, c)

fixed_keyword_starred_unpacking_func(**{'a': 1, 'b': 2, 'c': 3})

1 2 3


赋值语句中，`*expression`同样支持嵌套的可迭代对象的unpacking操作。

In [19]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *prices, (*_, year) = record
prices

[50, 123.45]

---

## 函数参数列表

函数的定义与调用是`*expression`与`**expression`一个重要的应用场景。

### 函数定义中的变长形参列表的形式

python函数定义的EBNF表达式(注意，官方文档中的EBNF表达式并没有完全表明函数定义语法，如参数默认值的限制条件便没有表示出来，此处还需要阅读文档中的解释文字)：

    /* 函数定义 */
    funcdef                 ::=  [decorators] "def" funcname "(
                                   [parameter_list] ")" ["->" expression] ":" suite
    
    /* 装饰器 */
    decorators              ::=  decorator+
    decorator               ::=  "@" dotted_name ["(" [argument_list [","]] ")"] NEWLINE
    dotted_name             ::=  identifier ("." identifier)*
    
    /* 参数列表 */
    parameter_list          ::=  defparameter ("," defparameter)* ["," [parameter_list_starargs]]
                                 | parameter_list_starargs
    parameter_list_starargs ::=  "*" [parameter] ("," defparameter)* ["," ["**" parameter [","]]]
                                 | "**" parameter [","]
    parameter               ::=  identifier [":" expression]
    defparameter            ::=  parameter ["=" expression]
    funcname                ::=  identifier
    
python官方文档通过EBNF表达式提供了函数定义的语法，在抛开函数装饰器、函数参数注解和函数返回值注解后，可以将函数定义中的形参列表进行简化，并分为四个部分。

    funcname(def_positional_parameters, *args, def_keyword_parameters, **kwargs):
        pass

即
1. `def_positional_parameters`，如`func(a, b, c, ...)`
2. `*args`，如`func(..., *args, ...)`
3. `def_keyword_parameters`，如`func(..., *, x, y, z, ...)`
4. `**kwargs`，如`func(..., *, ..., **kwargs)`

这四部分形参列表的解释与注意事项如下：

- `def_positional_parameters`和`def_keyword_parameters`形式相同，都为包含了任意数量的给定了参数名的参数，其中又分为未给定默认值的参数（`identifier`）和给定了默认值的参数（`identifier=expression`），且未给定默认值的参数一定位于给定了默认值的参数之前，这两类参数同样都是可选的。
- `def_positional_parameters`只负责接收positional参数，而`def_keyword_parameters`只负责接收keyword参数。
- 形如`*args`的参数至多出现一次，且要求一定出现在`def_positional_parameters`之后，同时要求出现在`def_keyword_parameters`或`**kwargs`参数之前，负责接收变长的positional参数。
- 形如`**kwargs`的参数至多出现一次，且一定位于参数列表中所有形参的最末尾，负责接收变长的keyword参数。
- 在函数体中使用变量`args`和`kwargs`访问传入的可变变量，变量`args`的类型为`tuple`，而变量`kwargs`类型为`dict`类型。

In [6]:
def var_positonal_and_keywords_parameters_func(a, b, c, *args, d, e, f, **kwargs):
    print(a, b, c)
    print(args)
    print(d, e, f)
    print(kwargs)
    
var_positonal_and_keywords_parameters_func(1, 2, 3, 4, 5, d=10, e=20, f=30, h=40, i=50)

1 2 3
(4, 5)
10 20 30
{'i': 50, 'h': 40}


这一特性与“赋值语句中左值变量的unpacking操作”类似，`*expression`变量和`**expression`变量可用于接受变长的参数，不同的是函数中`*args`变量最终为`tuple`类型，而在左值变量赋值语句中变量最终为`list`类型；同时，函数定义中，形参列表可以`*expression`变量和`**expression`变量两种变量，而在赋值语句中只有`*expression`变量。

可以将函数形参视为赋值语句中的左值变量，而实参为右值表达式，同样的，`*expression`变量和`**expression`变量可以接受变长的参数列表。

如果需要定义一个函数，要求该函数只接受keyword参数，则需要使用如下定义形式，其中所有在`*`和`*args`之后的参数都是仅接受keyword实参的形参。

In [27]:
# funcname(*, def_keyword_parameters, **kwargs)
def keyword_parameters_only_func(*, a, b, c, **kwargs):
    print(a)
    print(b)
    print(c)
    print(kwargs)

params = dict(a=1, b=2, c=3, d=4, e=5)
keyword_parameters_only_func(**params)

1
2
3
{'e': 5, 'd': 4}


#### 函数参数默认值为可变对象

python中，函数在定义时会将当前的命名空间中的该函数名绑定到一个函数对象上，这种绑定操作只会执行一次。因此在函数定义时，给定了默认值的参数也只会在函数定义时绑定一次默认值，而在之后的调用中，函数都将使用默认值所指向的同一个对象。因此在函数参数绑定的默认值为一个可变对象时，这一特性需要引起注意。可变对象包括`list`、`set`和`dict`对象等。

In [3]:
def default_value_binding(val, values=[]):
    values.append(val)
    return values

res = default_value_binding(1)
print(res)
res = default_value_binding(2)
print(res)
res = default_value_binding(3)
print(res)

[1]
[1, 2]
[1, 2, 3]


## 函数调用中的实参列表形式
    
python函数调用的EBNF表达式：
    
    call                 ::=  primary "(" [argument_list [","] | comprehension] ")"
    
    argument_list        ::=  positional_arguments ["," starred_and_keywords]
                                ["," keywords_arguments]
                              | starred_and_keywords ["," keywords_arguments]
                              | keywords_arguments
    positional_arguments ::=  ["*"] expression ("," ["*"] expression)*
    starred_and_keywords ::=  ("*" expression | keyword_item)
                              ("," "*" expression | "," keyword_item)*
    keywords_arguments   ::=  (keyword_item | "**" expression)
                              ("," keyword_item | "," "**" expression)*
    keyword_item         ::=  identifier "=" expression
    
对函数调用语法的EBNF表达式简化，只关注函数调用中的实参列表，可以将其分为两个部分：

    funcname(positional_parameters, keyword_parameters)
    
即

1. `positional_parameters`，如`func(1, 2, 3, ...)`
2. `keyword_parameters`，如`func(..., a=1, b=2, c=3)`
    
这两部分实参列表的解释与注意事项如下：

- `positional_parameters`是指直接提供实参值的形式，如`expression`；而`keyword_parameters`是指提供形参变量与实参值的形式，如`parameter=expression`。
- `positional_parameters`为函数定义中的`def_positional_parameters`和`*args`提供参数；而`keyword_parameters`为函数定义中的`def_keyword_parameters`和`**kwargs`提供参数。

In [7]:
def func(a, b, c=1, *args, d, e, f=10, **kwargs):
    pass

func(1, 2, 3, 4, 5, d=6, e=7, f=8, g=9, h=10)

### 函数调用中可迭代实参变量的unpacking操作

当需要将变长的可迭代对象作为实参传递给函数参数时，就需要用到`*expression`和`**expression`。

`*expression`和`**expression`常见的一个应用是对`tuple`、`list`或`dict`等变量进行unpacking操作，在转换后作为实参变量传递给函数的positional参数和keyword参数。

这一特性与“`list`、`tuple`、`set`以及dict中的可迭代对象的unpacking操作”类似
。可以将包含了可迭代对象的函数实参列表视为赋值语句中的一个右值表达式，将函数的形参列表视为赋值语句中多个等待被赋值的左值变量。此时通过`*expression`和`**expression`，将实参中的可迭代对象进行unpacking操作，最后将经过unpacking的实参列表传递给函数形参列表。

可迭代对象类型的实参通过unpacking操作转换后传递给函数形参列表。

#### `*expression`与positional参数

`*expression`可用于对函数参数列表中的可迭代对象实参进行unpacking操作，包括`tuple`、`list`、`string`、`file`，其最后返回的是一组positional函数实参。

In [24]:
def starred_unpacking_func(a, b, *args):
    print(a, b, args)
    
starred_unpacking_func(*(1, 2, 3))
starred_unpacking_func(*[1, 2, 3])
starred_unpacking_func(*(1, 2, 3), *[1, 2, 3])

1 2 (3,)
1 2 (3,)
1 2 (3, 1, 2, 3)


#### `**expression`与keyword参数

`**expression`可用于对函数参数列表中的`dict`类型的实参进行unpacking操作，其最后返回的是一组keyword函数实参。

In [26]:
def starred_starred_unpacking_func(*, a, b, **kwargs):
    print(a, b, kwargs)
    
starred_starred_unpacking_func(a=1, b=2)
starred_starred_unpacking_func(**dict(a=1, b=2, c=3))
starred_starred_unpacking_func(**dict(a=1, b=2, c=3), **dict(d=4, e= 5))

1 2 {}
1 2 {'c': 3}
1 2 {'c': 3, 'e': 5, 'd': 4}


需要注意的是，`*expression`和`**expression`并非只为变长参数提供实参，对于定长的positional参数和keyword参数同样适用。

In [12]:
def fixed_positional_starred_unpacking_func(a, b, c):
    print(a, b, c)

fixed_positional_starred_unpacking_func(*[1, 2, 3])

1 2 3


In [13]:
def fixed_keyword_starred_unpacking_func(*, a, b, c):
    print(a, b, c)

fixed_keyword_starred_unpacking_func(**{'a': 1, 'b': 2, 'c': 3})

1 2 3
